In [14]:
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten
from fit_generator import SpectogramGenerator
from math import floor

In [15]:
'''
It is possible to provide my own metric, the classic metric is accuracy... 

I will have to wrap up some mir_eval functionality into a function which I will pass on to the model at the compilation step. 
Okay in order for this to work I think.. that I am going to have to add an extra dimension to the X_training data.... so that.. 

'''
num_epochs = 10
input_dimensions = (5,1025,1)

In [ ]:
def load_train_data(dimension=(5,1025)):
    
    X_train = np.zeros((382182,*dimension))
    y_train = np.zeros((382182,48))
    current_spectogram_index = 0
    for i in range(291):
        spec,annotation = load_transform_and_annotation(i,train=True)
        X_train[current_spectogram_index:current_spectogram_index+spec.shape[0]] = spec
        y_train[current_spectogram_index:current_spectogram_index+spec.shape[0]] = annotation
                       
        current_spectogram_index += spec.shape[0]
    return X_train, y_train
                      
def load_test_data(dimension=(5,1025)):
    X_test = np.zeros((5, *dimension, 88281))
    y_test = np.zeros((48, 88281))
    current_spectogram_index = 0
                      
    for i in range(291,361):
        spec,annotation = load_transform_and_annotation(id_,train=False)
        X_test[current_spectogram_index:current_spectogram_index+spec.shape[0]] = spec
        y_test[current_spectogram_index:current_spectogram_index+spec.shape[0]] = annotation
                      
        current_spectogram_index += spec.shape[0]
    return X_test, y_test
                      

In [ ]:
def load_transform_and_annotation(id_, train = True, spectogram = 'fft'):
    if train:
        path = 'data/spec_labels/train/' + 'fileid_' + str(id_) + '/'
    else:
        path = 'data/spec_labels/test/'+'fileid_' + str(id_) + '/'
    annotation_label = np.load(path+'annotation_label.npy')

    if spectogram == 'fft':
        spec = np.load(path+'stft.npy')

        X_train = generate_windowed_samples(spec)

        return X_train, annotation_label
    elif spectogram == 'cqt':
        cqt = np.load(path+'cqt.npy')
        X_train = generate_windowed_samples(cqt)
        return X_train, annotation_label
    
def generate_windowed_samples(spec):
    '''
    Alright I believe that we will be having windows... 

    '''
    windowed_samples =np.zeros((spec.shape[0],5, spec.shape[1]))
    for i in range(spec.shape[0]):
            windowed_samples[i] = generate_sample(spec,i)

            ## The pointer logic would go here it seems like... 
            #windowed_samples[i] = spec[i -2:i +3]
    return windowed_samples

def generate_sample(spec, time_index):
    '''
    This function will two take an index... 

    '''

    if time_index <= 1:
        slice_window = np.zeros((5, spec.shape[1]))
    elif time_index >= spec.shape[0]-2:
        slice_window = np.zeros((5, spec.shape[1]))

    else:
        slice_window = spec[time_index-2:time_index+3]
    return slice_window

In [ ]:
X_train, y_train = load_train_data()
X_train = X_train.reshape((*X_train.shape,1))

In [17]:
generator = SpectogramGenerator()
valgenerator = SpectogramGenerator(train=False)

In [18]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (3,3), kernel_initializer='normal', activation='relu', padding = 'same',input_shape=( 5,1025,1)))
model.add(MaxPool2D(pool_size =(2,2)))
model.add(Dropout(.25))
model.add(Flatten())
model.add(Dense(128, activation='tanh'))
model.add(Dropout(.2))
model.add(Dense(48,kernel_initializer='normal', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics =['accuracy'])

batch_size = 32

In [19]:
model.fit_generator(generator,
                      epochs=num_epochs,
                      steps_per_epoch = floor(382182/batch_size),
                      verbose=1,
                      use_multiprocessing=True,
                      workers=16,
                      validation_data = valgenerator,
                      validation_steps = floor(88281/batch_size),
                      max_queue_size=32)

Epoch 1/10


/home/mddarr/anaconda3/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


   22/11943 [..............................] - ETA: 1:16:17 - loss: 0.2575 - acc: 0.9538

IndexError: pop from empty list